# Advent of Code 2022: Day 11
https://adventofcode.com/2022/day/11


## Part 1
Find the product of the number of items inspected
by the two monkeys that inspected the most number of items

### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')

### Define helper functions

#### Function to preprocess the data

In [2]:
def pre_process_data(datalist):
  
  # Setup a dictionary
  # and a counter
  monkeys = {}
  monkey_counter = 0

  # Go through the items
  # in the datalist
  for i in range(len(datalist)):
    
    # If the current item contains
    # "Monkey" this indicates that
    # a monkey has been encountered
    if "Monkey" in datalist[i]:
      
      # Item i+1 will contain the
      # items that the monkey is holding.
      # Split this on empty spaces and
      # ignore the first 4 objects.
      items = datalist[i+1].split(' ')[4:]
      
      # Convert the items to integers,
      # removing ',' if necessary. 
      for j in range(len(items)):
        items[j] = int(items[j].rstrip(','))
      
      # Item i+2 will contain the operation.
      # Split this on empty spaces and
      # ignore the first 5 objects.
      operation = datalist[i+2].split(' ')[5:]

      # Item i+3 will contain the test.
      # Split this on empty spaces and
      # take the 5th object. 
      test = int(datalist[i+3].split(' ')[5])

      # Items i+4 and i+5 will contain
      # the outcome. Split each item
      # on empty spaces and make a list
      # with the 9th item from each.
      outcome = [datalist[i+4].split(' ')[9], datalist[i+5].split(' ')[9]]

      # Put all of the obtained information 
      # in a dictionary, and place this
      # inside the main dictionary with
      # the current monkey as the key.
      monkeys["Monkey " + str(monkey_counter)] = {"Items": items,
                                                  "Operation": operation,
                                                  "Test": test,
                                                  "Outcome": outcome,
                                                  "Inspection_counter" : 0}
      monkey_counter +=1
  return monkeys

#### Function to create an item list if needed

In [3]:
def create_itemlist(monkeys):
  
  # Setup the itemlist
  # as a dictionary,
  # along with some
  # temporary values.
  item_list = {}
  temp_list = []
  j = 0

  # Go through each monkey
  for m in monkeys:

    # Add the current monkey
    # as a key in the item_list,
    # starting as an empty list.
    item_list[m] = []
    
    # Go through each item for a monkey
    for i in range(len(monkeys[m]["Items"])):
      # Add the current item to
      # the temporary list
      temp_list.append(monkeys[m]["Items"][i])
      
      # Replace the current item
      # with its index. 
      monkeys[m]["Items"][i]=j
      
      # Increase the index
      j+=1
  
  # For each monkey, add a 
  # copy of the temporary list
  for m in item_list:
    item_list[m]=temp_list.copy()
  return monkeys, item_list

#### Operation carried out by a monkey

In [4]:
def operation(item, op, worry_denominator):
  
  # Get a copy of the operation
  t_op = op.copy()
  
  # Go through each part
  # of the operation
  for k in range(len(t_op)):
    
    # If a part is equal to
    # "old" replace it with
    # the actual item.
    if t_op[k] == "old":
      t_op[k] = item

  # Get the two numerical
  # parts of the operation,
  # make sure to convert
  # the second one in case
  # it is still a string.
  one = t_op[0]
  two = int(t_op[2])

  # Return a value based on the
  # middle part of the operatio.
  if t_op[1]=='+':
    return int((one+two)/worry_denominator)
  elif t_op[1] == "*":
    return int((one*two)/worry_denominator)

#### Check where an item should be thrown

In [5]:
def check_item(monkeys, curr_monkey, curr_op, curr_test, worry_denominator):
  
  # Pop the first item from the current monkey
  curr = monkeys[curr_monkey]["Items"].pop(0)
  
  # Perform the operation on the resulting value
  curr = operation(curr, curr_op, worry_denominator)
  
  # Perform the test on the item
  if(curr % curr_test) == 0:
    return True, curr
  else:
    return False, curr

#### Check where an item should be thrown, using the item list

In [6]:
def check_item_with_list(monkeys, curr_monkey, curr_op, curr_test, worry_denominator, itemlist):
  
  # Pop the first id from the current monkey
  curr_id = monkeys[curr_monkey]["Items"].pop(0)
  
  for m in monkeys:
    
    # Perform the current operation on the item
    # at the current id for each monkey.
    curr_value = operation(itemlist[m][curr_id], curr_op, worry_denominator)
    
    # Perform the test on the resulting value
    # for each monkey.
    curr_value = curr_value % monkeys[m]["Test"] 
    
    # Add the result to the item list
    itemlist[m][curr_id] = curr_value

  # Check what should be returned.
  if itemlist[curr_monkey][curr_id] == 0:
    return True, curr_id
  else:
    return False, curr_id

#### Have a monkey take a turn

In [7]:
def monkey_turn(monkeys, index, worry_denominator, s=False, itemlist = None):
  
  # Get the current values
  # based on the index.
  curr_monkey = "Monkey " + str(index)
  curr_op = monkeys[curr_monkey]["Operation"]
  curr_test = monkeys[curr_monkey]["Test"]
  curr_outcome = monkeys[curr_monkey]["Outcome"]
  n = len(monkeys[curr_monkey]["Items"])

  # Go through each item
  for i in range(n):
    
    # Call the appropriate check, based on
    # if the safety to prevent overflow is on
    if(s == True):
      test, c = check_item_with_list(monkeys, 
                                     curr_monkey, 
                                     curr_op, 
                                     curr_test, 
                                     worry_denominator, 
                                     itemlist)    
    else:
      test, c = check_item(monkeys,
                           curr_monkey, 
                           curr_op, 
                           curr_test, 
                           worry_denominator)
    
    # Get the outcome
    # based on the check result
    if(test):
      throw_to = curr_outcome[0]
    else:
      throw_to = curr_outcome[1]

    # Get the target monkey   
    target_monkey = "Monkey " + throw_to
    
    # Append the value returned from 
    # the check to the target monkey
    monkeys[target_monkey]["Items"].append(c)
    monkeys[curr_monkey]["Inspection_counter"] += 1
  return monkeys

#### Have the monkeys complete a round

In [8]:
def monkey_round(monkeys, worry_denominator,s = False, itemlist=None):
  for m in range(len(monkeys)):
    monkeys = monkey_turn(monkeys, m, worry_denominator,s, itemlist)
  return monkeys

### Main function

In [9]:
"""Find the product of the number of items inspected 
   by the two monkeys that inspected the most number of items 

@param datalist: The data (a list of strings)
@param rounds: The number of rounds (default is 1)
@param worry_denominator: The value to divide the items
                          by in every operation (default is 3)
@param s: If the function should safeguard against overflow (default is false)
"""

def monkey_business(datalist, rounds = 1, worry_denominator = 3, s = False):
  
  # Get the monkey dictionary
  # by preprocessing the data
  monkeys = pre_process_data(datalist)
  item_list = None
  
  # If the safety is on,
  # create the item list
  if s == True:
    monkeys, item_list = create_itemlist(monkeys)
  
  # Carry out the given
  # number of rounds
  for r in range(rounds):
    monkeys = monkey_round(monkeys, worry_denominator, s, item_list)

  # Get the number of items
  # each monkey has inspected
  # after all the rounds.
  inspection_list = []
  for m in monkeys:
    inspection_list.append(monkeys[m]["Inspection_counter"])
  
  # Sort the list and return the
  # product of the 2 largest values.
  inspection_list.sort()
  return inspection_list[-1]*inspection_list[-2]

In [10]:
monkey_business(data_list, 20)

100345

## Part 2
Find the product of the number of items inspected
by the two monkeys that inspected the most number of items.

No decay of worries (worry_denominator = 1) and risk of overflow.

In [11]:
monkey_business(data_list, 10000, 1, True)

28537348205